# Default Parameters

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

dataset = 2 # 1 or 2

# Load Dataset

In [2]:
from pyspark.sql import functions as F

# if the option is set as 1 load bank dataset
if dataset == 1:
    filename = "bank-full.csv"
    target_variable_name = 'y'
    df = spark.read.csv(filename, header=True, inferSchema=True, sep=';')
    df = df.withColumn(target_variable_name, F.when(df[target_variable_name] == 'no', 0).otherwise(1))
# if the option is not set as 1 load housing dataset
else:
    filename = "melb_data.csv"
    target_variable_name = "type"
    df = spark.read.csv(filename, header=True, inferSchema=True, sep=',')
    df = df.withColumn(target_variable_name, F.when(df[target_variable_name] == 'h', 0).otherwise(1))

df.show()

+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|type|    Price|Method|SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|       85 Turner St|    2|   0|1480000.0|     S| Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford|    25 Bloomburg St|    2|   0|1035000.0|     S| Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0| 

In [3]:
df.groupby(target_variable_name).count().show()

+----+-----+
|type|count|
+----+-----+
|   1| 4131|
|   0| 9449|
+----+-----+



In [4]:
df.dtypes

[('Suburb', 'string'),
 ('Address', 'string'),
 ('Rooms', 'int'),
 ('type', 'int'),
 ('Price', 'double'),
 ('Method', 'string'),
 ('SellerG', 'string'),
 ('Date', 'string'),
 ('Distance', 'double'),
 ('Postcode', 'double'),
 ('Bedroom2', 'double'),
 ('Bathroom', 'double'),
 ('Car', 'double'),
 ('Landsize', 'double'),
 ('BuildingArea', 'double'),
 ('YearBuilt', 'double'),
 ('CouncilArea', 'string'),
 ('Lattitude', 'double'),
 ('Longtitude', 'double'),
 ('Regionname', 'string'),
 ('Propertycount', 'double')]

# Weight of Evidence

In [5]:
#identify variable types and perform some operations
def variable_type(df):
    
    vars_list = df.dtypes
    char_vars = []
    num_vars = []
    for i in vars_list:
        if i[1] in ('string'):
            char_vars.append(i[0])
        else:
            num_vars.append(i[0])
    
    return char_vars, num_vars

In [6]:
char_vars, num_vars = variable_type(df)

if dataset != 1:
    char_vars.remove('Address')
    char_vars.remove('SellerG')
    char_vars.remove('Date')
    char_vars.remove('Suburb')

In [7]:
num_vars.remove(target_variable_name)

In [8]:
final_vars = char_vars + num_vars

In [9]:
# WOE & IV code
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import VectorAssembler
import scipy.stats.stats as stats

#default parameters
#rho value for spearman correlation. You can adjust this value
custom_rho = 1
#maximum number of bins to start of with. It will keep decreasing from this number. Adjustable parameter
max_bin = 20 

# This function calculates the WOE and IV values based on the Pyspark output. 
# Note: This portion of the code is python based implementation for efficieny purpose
def calculate_woe(count_df, event_df, min_value, max_value, feature):
    
    # implemeting the table structure shown in WOE & IV calculation figure
    woe_df = pd.merge(left=count_df, right=event_df)
    woe_df['min_value'] = min_value
    woe_df['max_value'] = max_value
    woe_df['non_event'] = woe_df['count'] - woe_df['event']
    woe_df['event_rate'] = woe_df['event']/woe_df['count']
    woe_df['nonevent_rate'] = woe_df['non_event']/woe_df['count']
    woe_df['dist_event'] = woe_df['event']/woe_df['event'].sum()
    woe_df['dist_nonevent'] = woe_df['non_event']/woe_df['non_event'].sum()
    woe_df['woe'] = np.log(woe_df['dist_event']/woe_df['dist_nonevent'])
    woe_df['iv'] = (woe_df['dist_event'] - woe_df['dist_nonevent'])*woe_df['woe']
    woe_df['varname'] = [feature]* len(woe_df)
    woe_df = woe_df[['varname','min_value', 'max_value', 'count', 'event', 'non_event', 'event_rate', 'nonevent_rate', 'dist_event','dist_nonevent','woe', 'iv']]
    woe_df = woe_df.replace([np.inf, -np.inf], 0)
    woe_df['iv'] = woe_df['iv'].sum()
    return woe_df

#monotonic binning function implemented along with spearman correlation
def mono_bin(temp_df, feature, target, n = max_bin):
    
    r = 0
    while np.abs(r) < custom_rho and n > 1:
    
        try:
            #Quantile discretizer cuts data into equal number of observations
            qds = QuantileDiscretizer(numBuckets=n, inputCol=feature, outputCol='buckets', relativeError=0.01)
            bucketizer = qds.fit(temp_df)
            temp_df = bucketizer.transform(temp_df)

            #create corr_df is Python based implementation for efficiency purpose
            corr_df = temp_df.groupby('buckets').agg({feature:'avg', target:'avg'}).toPandas()
            corr_df.columns = ['buckets', feature, target]
            r, p = stats.spearmanr(corr_df[feature], corr_df[target]) #spearman correlation
            n = n - 1 
        except Exception as e:
            n = n - 1
        
        return temp_df

#execute woe for all the variables in the dataset
def execute_woe(df, target):
    
    count = -1    
    for feature in final_vars:
        #execute if the feature is not a target column name. Provided as an extra check.
        if feature != target: 
            count = count + 1
            temp_df = df.select([feature, target])
            
            #perform monotonic binning for numeric variables before doing woe calculation
            if feature in num_vars:
                temp_df = mono_bin(temp_df, feature, target, n = max_bin)
                # group buckets in numerical
                grouped = temp_df.groupby('buckets')
            else:
                # just group categories in categorical
                grouped = temp_df.groupby(feature)
            
            #count and event value for each group
            count_df = grouped.agg(F.count(target).alias('count')).toPandas()
            event_df = grouped.agg(F.sum(target).alias('event')).toPandas()
            
            #store min and max values for variables. for category both takes the same value.
            if feature in num_vars:
                min_value = grouped.agg(F.min(feature).alias('min')).toPandas()['min']
                max_value = grouped.agg(F.max(feature).alias('max')).toPandas()['max']
            else:
                min_value = count_df[feature]
                max_value = count_df[feature]
            
            #calculate woe and iv
            temp_woe_df = calculate_woe(count_df, event_df, min_value, max_value, feature)
            
            #final dataset creation
            if count == 0:
                final_woe_df = temp_woe_df
            else:
                final_woe_df = final_woe_df.append(temp_woe_df, ignore_index=True)
        
        # separate IV dataset creation
        iv = pd.DataFrame({'IV':final_woe_df.groupby('varname').iv.max()})
        iv = iv.reset_index()
    return final_woe_df, iv

In [10]:
# invoke woe & iv code
output, iv = execute_woe(df, target_variable_name)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
output

varname min_value max_value  count  event  non_event  event_rate  \
0           Method        PI        PI   1564    495       1069    0.316496   
1           Method        SA        SA     92     26         66    0.282609   
2           Method        SP        SP   1703    624       1079    0.366412   
3           Method        VB        VB   1199    471        728    0.392827   
4           Method         S         S   9022   2515       6507    0.278763   
..             ...       ...       ...    ...    ...        ...         ...   
238  Propertycount      4217      4794    759    229        530    0.301713   
239  Propertycount     10175     10894    672    207        465    0.308036   
240  Propertycount      5943      6482    622    185        437    0.297428   
241  Propertycount     10926     11204    532    137        395    0.257519   
242  Propertycount      7570      8256    692    246        446    0.355491   

     nonevent_rate  dist_event  dist_nonevent       woe        iv  
0         0.683504    0.119826       0.113134  0.057468  0.035096  
1         0.717391    0.006294       0.006985 -0.104169  0.035096  
2         0.633588    0.151053       0.114192  0.279750  0.035096  
3         0.607173    0.114016       0.077045  0.391946  0.035096  
4         0.721237    0.608811       0.688644 -0.123216  0.035096  
..             ...         ...            ...       ...       ...  
238       0.698287    0.055435       0.056091 -0.011766  0.071739  
239       0.691964    0.050109       0.049212  0.018071  0.071739  
240       0.702572    0.044783       0.046248 -0.032188  0.071739  
241       0.742481    0.033164       0.041803 -0.231515  0.071739  
242       0.644509    0.059550       0.047201  0.232402  0.071739  

[243 rows x 12 columns]

In [17]:
iv.sort_values('IV', ascending=False)

varname        IV
6        Landsize  2.835305
14          Rooms  1.750996
1        Bedroom2  1.633744
11          Price  1.551814
2    BuildingArea  0.884353
15      YearBuilt  0.851490
3             Car  0.843557
4     CouncilArea  0.569996
5        Distance  0.340613
10       Postcode  0.270258
8      Longtitude  0.259286
7       Lattitude  0.229285
13     Regionname  0.195509
0        Bathroom  0.166885
12  Propertycount  0.071739
9          Method  0.035096